## Download entire block model

In [ ]:
# Import modules

import json
from http import HTTPStatus

import pandas as pd
import requests

In [ ]:
# Enter your access token, Evo hub URL, org ID, workspace ID and block model ID.

access_token = ""
evo_hub_url = ""
org_id = ""
workspace_id = ""
block_model_id = ""

auth_header = {"Authorization": "Bearer " + access_token}

In [ ]:
# Compose the URL, query parameters and JSON body
url = f"{evo_hub_url}/blockmodel/orgs/{org_id}/workspaces/{workspace_id}/block-models/{block_model_id}/blocks"

body = {"columns": ["*"], "geometry_columns": "coordinates"}

# Make a POST request and include the JSON body and headers
response = requests.post(url=url, json=body, headers=auth_header)
response_output = json.dumps(response.json(), indent=4, sort_keys=True)

# Copy the `job_url` parameter from the response
if response.status_code == HTTPStatus.OK:
    print(response_output)
    job_url = response.json()["job_url"]
else:
    raise Exception(f"Request failed: \n Status: {response.status_code} \n Response: {response_output}")

In [ ]:
# Keep checking the `job_url` until the response gives you a `download_url` parameter

# Make a GET request, including the headers
response = requests.get(url=job_url, headers=auth_header)
response_output = json.dumps(response.json(), indent=4, sort_keys=True)
print(response_output)
if response.json()["job_status"] == "COMPLETE":
    download_url = response.json()["payload"]["download_url"]

In [ ]:
# Download the data

# Make a GET request
response = requests.get(url=download_url)

# Save the file
open("query_results.parquet", "wb").write(response.content)

# Display the first rows of data
df = pd.read_parquet("query_results.parquet")
df.head()